# Comparing other feature selection methods with triku
In this notebook we will compare the performance of triku, compared to other methods. 

The methods that will be compared will be the following:
* Select genes with highest variance.   
* Scanpy's `sc.pp.highly_variable_genes`: It is based on Seurat's `vst` method, so they should return similar results.
* scry `devianceFeatureSelection()`. This method is featured as the feature selection for Irizarry's GLM-PCA paper (https://doi.org/10.1186/s13059-019-1861-6). From its description, it computes a deviance statistic for each row feature for count data based on a multinomial null model that assumes each feature has a constant rate. Features with large deviance are likely to be informative. Uninformative, low deviance features can be discarded to speed up downstream analyses and reduce memory footprint. The `fam`parameter will be set to `binomial`, the default.
* M3Drop, which has two main functions:
    * NBDrop: the NBDrop model assumes proportion of zeros follows a Michaelis-Menten model. Then the Michaelis-Menten parameter $K$ is fitted. For each gene, its parameter $K_i$ is compared to $K$ using a $Z$-test, which returns the selected genes.
    * NBUmi: The procedure is similar to above, although the equation to fit now is a negative binomial model,  and the selection of genes is then done using a $Z$-test.
* `BrenneckeGetVariableGenes` fits a function between CV$^2$ and mean expression. 

With the exception of scanpy and triku, the rest of functions are set on $R$. We will use jupyter's `%%R` magic command, and `anndata2ri` to transform `annData` into `SingleCellExperiment` objects, and we will generate the functions to accept that annData and return the list of selected features. The functions have to be set up in notebook, and cannot be externalized. 

M3Drop requires a normalization step, which will be done in-situ.

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import triku as tk
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.sparse as spr
import scipy.stats as sts
import os
import gc
from itertools import product
import pickle
import ray
import seaborn as sns

from IPython.display import display, HTML

from tqdm.notebook import tqdm

from bokeh.io import show, output_notebook, reset_output
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D

from sklearn.metrics import adjusted_rand_score as ARS
from sklearn.metrics import adjusted_mutual_info_score as NMI
from sklearn.metrics import silhouette_score, davies_bouldin_score

reset_output()
output_notebook()

In [ ]:
!python setup.py install

In [ ]:
import sys, os
sys.path.insert(0, os.getcwd() + '/code')

from triku_nb_code.comparing_feat_sel import plot_max_var_x_dataset, plot_max_var_x_method, create_dict_UMAPs_datasets, \
get_max_diff_gene, plot_ARI_x_method, plot_ARI_x_dataset, biological_silhouette_ARI_table, plot_lab_org_comparison_scores, \
clustering_binary_search, compare_rankings, compare_values
from triku_nb_code.comparing_feat_sel import create_UMAP_adataset_libprep_org, plot_UMAPs_datasets, plot_XY, biological_silhouette_ARI_table
from triku_nb_code.palettes_and_cmaps import magma, bold_and_vivid, prism

In [ ]:
%matplotlib inline

In [ ]:
import rpy2.rinterface_lib.callbacks, logging
from rpy2.robjects import pandas2ri
import anndata2ri

In [ ]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
anndata2ri.activate()
pandas2ri.activate()
%load_ext rpy2.ipython
%load_ext rmagic

In [ ]:
%%R
# Load all the R libraries we will be using in the notebook
library(M3Drop) # Depends on r-foreing (conda-forge) and Hmisc and reldist (install.packages)
library(scry) # If R < 4, launch commit 9f0fc819

In [ ]:
os.makedirs(os.getcwd() + '/exports/comparisons/', exist_ok=True)
os.makedirs(os.getcwd() + '/figures/comparison_figs/png', exist_ok=True)
os.makedirs(os.getcwd() + '/figures/comparison_figs/pdf', exist_ok=True)
data_dir = os.path.dirname(os.getcwd()) + '/data/'

In the following 3 cells we will create the cells that obtain the most relevant features. Since some of the calls are to R, they have to be kept as separate cells. Also, we create the function `create_df_feature_ranking` which creates two dataframes: one with the evaluation values (p-value, emd distance, etc.) of each method, and the second one with the ranking of genes based on those values. These dataframes will be valuable so that we don't have to repeat the calling to the feature selection methods each time we do a graph. `create_df_feature_ranking` is also kept as a cell because it makes some calls to R.

In [ ]:
%%R

run_scry <- function(sce){ #adata
    adata_ret = devianceFeatureSelection(sce, nkeep=dim(sce)[1])
    return(adata_ret) #returns adata with stats on .var
} 


run_brennecke <- function(sce){ #df
    res_df <- BrenneckeGetVariableGenes(sce, suppress.plot=TRUE, fdr=100)
    return(res_df) # returns sorted df with genes and stats
}


run_M3Drop <- function(sce){
    norm <- M3DropConvertData(sce, is.counts=TRUE)
    DE_genes <- M3DropFeatureSelection(norm, suppress.plot=TRUE, mt_threshold=50)
    return(DE_genes) # returns sorted df with genes and stats
    
}

run_NBumi <- function(sce){
    count_mat <- NBumiConvertData(sce, is.counts=TRUE)
    DANB_fit <- NBumiFitModel(count_mat)
    NBDropFS <- NBumiFeatureSelectionCombinedDrop(DANB_fit, suppress.plot=TRUE, qval.thresh=10)
    return(NBDropFS)  # returns sorted df with genes and stats
    
}


In [ ]:
def run_scanpy(adata):
    adata_copy = adata.copy()
    if not 'log1p' in adata_copy.uns:
        sc.pp.log1p(adata_copy)
    ret = sc.pp.highly_variable_genes(adata_copy, n_top_genes=len(adata_copy), inplace=False)
    df = pd.DataFrame(ret)
    df =  df.set_index(adata_copy.var_names)
    del adata_copy; gc.collect()
    return df # returns df with stats

def run_variable(adata):
    if spr.issparse(adata.X):
        std = adata.X.power(2).mean(0) - np.power(adata.X.mean(0), 2) 
        std = np.asarray(std).flatten()        
    else:
        std = adata.X.std(0)
        
    return std #returns vector with order as var_names 

def run_triku(adata, seed):
    adata_copy = adata.copy()
    tk.tl.triku(adata_copy, n_comps=30, n_windows=100, random_state=seed, verbose='error')
    d = adata_copy.var['emd_distance'] #pd series with distance
    del adata_copy; gc.collect()
    return d

In [ ]:
def create_df_feature_ranking(adatax, title_prefix, apply_log=False):
    """
    Create a dataframe with the ranking of features, and another one with the feature values. The adata must be the raw
    adata. From that we will create a adata_df necessary for some R methods.
    The adata will include:
    - Triku with 10 seeds 'triku_SEEDN'
    - Scanpy's HVG 'scanpy'
    - Std 'std'
    - scry 'scry'
    - brennecke 'brennecke'
    - M3Drop 'm3drop'
    - NBumi 'nbumi'
    
    After each method is run, we will fill the dataframe values, with the values of the metrics used for feature selection, 
    and the dataframe of rankings with the rankings based on the returned value (0, 1, 2, etc.). 
    We create two separate dataframes because the df with values might be reserved for other purposes. The rank dataframes is interesting
    because the values on the values dataframe have different argsort orders depending on the column (M3drop and NBumi direct, rest reverse).
    """
    
    adata = adatax.copy()
    sc.pp.filter_genes(adata, min_cells=1) 
    sc.pp.filter_cells(adata, min_genes=1)
    
    if apply_log:
        sc.pp.log1p(adata)
    
    adata_df = pd.DataFrame(adata.X.T, index=adata.var_names, columns=adata.obs_names)
        
    adata_short = sc.AnnData(X = adata.X[:,:]) # we have to create a clean adata because some column break Rpush
    adata_short.var_names, adata_short.obs_names = adata.var_names[:], adata.obs_names[:]
    print(adata_short.shape)
    
    %Rpush adata_short
    %Rpush adata_df
    
    print('Outside R', adata.shape, adata_short.shape)
    d = %R  dim(adata_short)
    print('Inside R', d)
       
    if 'Group' in adata.obs:
        adata_groups = [i.replace('Group', '') for i in adata.obs['Group']]
        adata.obs['groupn'] = adata_groups
    
    index, columns = adata.var_names, [f'triku_{i}' for i in range(10)] + ['scanpy', 'std', 'scry', 'brennecke', 'm3drop', 'nbumi']
    df_values, df_ranks = pd.DataFrame(index=index, columns=columns), pd.DataFrame(index=index, columns=columns)
    
    for i in range(10):
        df_emd_distance = run_triku(adata, i)
        df_values.loc[df_emd_distance.index, f'triku_{i}'] = df_emd_distance.values
        
    
    scanpy_ret = run_scanpy(adata)
    df_values.loc[scanpy_ret.index, 'scanpy'] = scanpy_ret['dispersions_norm'].values
    assert len(df_values.index) == len(adata.var_names)
    
    std_ret = run_variable(adata)
    df_values.loc[:, 'std'] = std_ret
    assert len(df_values.index) == len(adata.var_names)
    
    scry_ret = %R run_scry(adata_short)
    df_values.loc[scry_ret.var.index, 'scry'] = scry_ret.var['binomial_deviance'].values
    assert len(df_values.index) == len(adata.var_names)
    
    brennecke_ret = %R run_brennecke(adata_df)
    df_values.loc[brennecke_ret.index, 'brennecke'] = brennecke_ret['effect.size'].values
    assert len(df_values.index) == len(adata.var_names)
    
    M3Drop_ret = %R run_M3Drop(adata_df)
    df_values.loc[M3Drop_ret.index, 'm3drop'] = M3Drop_ret['q.value'].values
    assert len(df_values.index) == len(adata.var_names)
    
    NBumi_ret = %R run_NBumi(adata_df)
    df_values.loc[NBumi_ret.index, 'nbumi'] = NBumi_ret['q.value'].values
    assert len(df_values.index) == len(adata.var_names)
    
    # Now we will fill df_ranks with an argsort !!!!! M3DROP and NBumi is not [::-1] because they are q-values 
    for col in [f'triku_{i}' for i in range(10)] + ['scanpy', 'std', 'scry', 'brennecke']:
        df_ranks[col] = df_values[col].values.argsort()[::-1].argsort()
    for col in ['m3drop', 'nbumi']:
        df_ranks[col] = df_values[col].values.argsort().argsort() # double argsort to return the rank!
    
    df_ranks.to_csv(os.getcwd() + '/exports/comparisons/' + title_prefix + '_feature_ranks.csv')
    df_values.to_csv(os.getcwd() + '/exports/comparisons/' + title_prefix + '_feature_values.csv')
    print('df_ranks', df_ranks.shape)
    
    del adata; gc.collect()
    return df_values, df_ranks

# Random datasets
For this section we will use the random datasets generated with splatter.
To evaluate the performance of the feature selection methods, we will use teo metrics, maximum deviation and ARI, explained below.

In [ ]:
splatter_dir = os.path.dirname(os.getcwd()) + '/data/splatter/'
list_deprobs = [0.005, 0.0065, 0.008, 0.01, 0.013, 0.016, 0.025, 0.05, 0.1, 0.3]

**THIS PROCESS TAKES ~ 5 HOURS!**

In [ ]:
# TODO Parallelize?
for deprob in tqdm(list_deprobs):
    print(f'NOW DEPROB IS {deprob}!!!!!!')
    adata_deprob = sc.read(splatter_dir + f'/splatter_deprob_{deprob}.loom', sparse=False)
        
    create_df_feature_ranking(adata_deprob, f'scatter_{deprob}')

## Maximum difference (or maximum variation)
We define maximum variation as the maximum value of the differences between groups. The maximum variation is calculated in the following steps:
* For each group in the dataset, and for each gene, select the gene if any group has more than X% of expressing cells (25% by default). If all groups have fewer than X% of expressing cells, the variation is set to 0. 
* If the gene is selected, calculate the trimmed mean (2.5% lowest and highest values removed by default) for each group.
* Scale the mean expression array to 1.
* Sort the values and select $\max(|a[0] - a[X]|, |a[-X] - a[-1]|)$ (X = 3 by default). We select first and last values because generally either one or two clusters are overexpressed, or one cluster is undeexpressed, and the expression values will appear at the beginning or end of the mean expression array.

In [ ]:
# Calculate the largest difference in all dataset
for deprob in tqdm(list_deprobs):
    adata_deprob = sc.read(splatter_dir + f'/splatter_deprob_{deprob}.loom', sparse=False)
        
    sc.pp.filter_genes(adata_deprob, min_cells=1)
    sc.pp.filter_cells(adata_deprob, min_genes=1)
    print(adata_deprob.shape)
    groups = sorted(list(dict.fromkeys(adata_deprob.obs['Group'].values)))
    df_max_var = pd.DataFrame(index=adata_deprob.var_names, columns=groups + ['maximum_variation'])
    
    for gene in tqdm(adata_deprob.var_names):
        max_var, arr_info = get_max_diff_gene(adata_deprob, gene, 'Group')
        df_max_var.loc[gene, groups] = arr_info
        df_max_var.loc[gene, 'maximum_variation'] = max_var
    
    print(df_max_var.shape)

    df_max_var.to_csv(os.getcwd() + '/exports/comparisons/' + f'scatter_{deprob}' + '_maximum_variation.csv')

We will plot two modalities:
* X axis represents the different feature selction methods, and each plot represents a dataset. We will select datasets with "conflictive" probabilities, like 0.01 or 0.016. For each feature selection methods, different features from 0-50, 50-100, 100-200, etc. are selected to show the stratification based on rank.
* X axis represents different datasets and, for each datasets, different feature selection methods are plotted. Each plot represents a number of features.

In [ ]:
deprob = 0.016

df_max_var = pd.read_csv(os.getcwd() + '/exports/comparisons/' + f'scatter_{deprob}' + '_maximum_variation.csv', index_col=0)
df_feature_ranks = pd.read_csv(os.getcwd() + '/exports/comparisons/' + f'scatter_{deprob}' + '_feature_ranks.csv', index_col=0)
df_feature_ranks = df_feature_ranks[['triku_0'] + [i for i in df_feature_ranks.columns if 'triku' not in i]].rename(columns = {'triku_0': 'triku'})

plot_max_var_x_method(df_feature_ranks, df_max_var, feature_list = [0, 50, 100, 200, 500, 1000], 
                      title=f'Maximum difference by number of features, random w/ DE {deprob}', 
                      file=f'max_diff_scatter_DE-{deprob}')

We see that, at least for triku, differences are more apparent for the first features (50 - 100), and for larger numbers of features the difference distribution diminishes. For features in range 0 to 100 or 200 triku shows the highest values, followed by scanpy and scry. Interestingly, brennecke fails to show any proper feature up to the first 200 features, which is bizarre.

In [ ]:
dict_df_feature_ranks, dict_df_max_var_dataset = {}, {}

for deprobx in [0.005, 0.0065, 0.008, 0.01, 0.013, 0.016, 0.025, 0.05, 0.1, 0.3]:
    df_max_var = pd.read_csv(os.getcwd() + '/exports/comparisons/' + f'scatter_{deprobx}' + '_maximum_variation.csv', index_col=0)
    df_feature_ranks = pd.read_csv(os.getcwd() + '/exports/comparisons/' + f'scatter_{deprobx}' + '_feature_ranks.csv', index_col=0)
    df_feature_ranks = df_feature_ranks[['triku_0'] + [i for i in df_feature_ranks.columns if 'triku' not in i]].rename(columns = {'triku_0': 'triku'})
    
    dict_df_feature_ranks[deprobx], dict_df_max_var_dataset[deprobx] = df_feature_ranks, df_max_var
    

plot_max_var_x_dataset(dict_df_feature_ranks, dict_df_max_var_dataset, n_features=100, title='Maximum difference in datasets, 100 features', 
                      file=f'max_diff_scatter_all-DE_100-features')
plot_max_var_x_dataset(dict_df_feature_ranks, dict_df_max_var_dataset, n_features=200, title='Maximum difference in datasets, 200 features', 
                      file=f'max_diff_scatter_all-DE_200-features')
plot_max_var_x_dataset(dict_df_feature_ranks, dict_df_max_var_dataset, n_features=500, title='Maximum difference in datasets, 500 features', 
                      file=f'max_diff_scatter_all-DE_500-features')
plot_max_var_x_dataset(dict_df_feature_ranks, dict_df_max_var_dataset, n_features=2500, title='Maximum difference in datasets, 2500 features', 
                      file=f'max_diff_scatter_all-DE_2500-features')

We see that, generally, triku shows the greatest difference across datasets, regardless of the number of features selected. For higher numbers of features the difference becomes less aparent at lower DE ranges(up to 0.025), simply because in those datasets the are not that many distinguishing features and, therefore, if more features are selected, they only contribute as *background noise*. 

To see why triku does perform better, we are going to choose a moderately difficult dataset (DE = 0.016), plot UMAPs for each method and its first features, and see any patterns that are missing.

In [ ]:
deprob = 0.016
adata_deprob = sc.read(splatter_dir + f'/splatter_deprob_{deprob}.loom', sparse=False)

sc.pp.filter_genes(adata_deprob, min_cells=1)
sc.pp.filter_cells(adata_deprob, min_genes=1)
    
tk.tl.triku(adata_deprob)
sc.pp.log1p(adata_deprob)
sc.pp.pca(adata_deprob, use_highly_variable=True)
sc.pp.neighbors(adata_deprob, n_neighbors=int(0.5 * len(adata_deprob) ** 0.5), metric='cosine')
sc.tl.umap(adata_deprob)

In [ ]:
n_rows = 60
fig, axs = plt.subplots(n_rows, 7, figsize=(7 * 3, n_rows * 2.5))

rank_list = list(np.arange(15)) + list(np.linspace(15, 150, n_rows-15).astype(int))

for col, method in enumerate(df_feature_ranks.columns):   
    for row in range(n_rows):
        gene = df_feature_ranks[method][df_feature_ranks[method] == rank_list[row]].index[0]
        max_var = df_max_var.loc[gene, 'maximum_variation']
        
        
        try:
            sc.pl.umap(adata_deprob, color=gene, cmap=magma, ax=axs[row][col], title='', show=False)
        except: 
            print(f'Fail in col {col} / row {row}')
            pass # sometimes it fails (failed once and dunno why)
        
        axs[row][col].set_xlabel(f"{gene} ({rank_list[row]}), {max_var:.2f}")
        axs[row][col].set_ylabel('')
        axs[row][col].xaxis.set_label_position('top') 
        axs[row][col].axes.get_xaxis().set_ticks([])
        axs[row][col].axes.get_yaxis().set_ticks([])
    
    
for col in range(7):
    axs[0][col].set_title(df_feature_ranks.columns[col])
    
    
plt.tight_layout()

There is not a clear reason why triku selects clusters with more differences. Generally, triku selects genes with appear underexpressed in one cluster, and does not favour as much the selection of genes that are expressed equally among clusters but unequally within each cluster. Therefore, by selecting genes overexpressed or underexpressed in one cluster, its scores are better; but this does not mean that other methods do not do also that. Hoever, judging by the difference scores, it seems that the rest of methods prefer genes that are overexpressed within each cluster rather than overexpressed in one cluster. Again, this is a trend, all methods select genes with overexpression in one cluster (at least the more apparent ones).

Brennecke and M3Drop completely fail here because the distributions that are fitted for the methods do not correspond to the ones belonging to the gene count matrix. This does not mean that they won't perform as badly in real datasets, but shows how distribution-fitting dependent methods are unstable in other types of datasets. We will evaluate this performance in the biological benchmarking datasets, to see if they underperform in other-than-usual library preparation methods.

## ARI / NMI
Using ARI on random datasets is a measure to assess the effectiveness of the feature selection. Random datasets were prepared with different degrees of differentially expressed gene probability, so that we can compare the leiden clusterign solution with the 9 populations. Triku can be run with different seeds, but the rest of methods are deterministic. However, leiden clustering in all cases can be run with a seed. Therefore, we are going to run all processes with 10 seeds (although the deterministic processes will be run once).

To apply the ARI we need to run leiden with as many clusters as scatter populations. Since leiden runs on resolution, we need to adjust the resolution parameter to match the number of clusters. To do that we are going to implement a binary search-like algorithm. We will start with resolutions 0.3 and 2 (may change in the future). If any of those yields the clusters, done. Else, find the midpoint, run the clustering, and if the clustering yields the number of populations, stop. Else, set the upper or lower resolution to the one that makes the desired number of clusters to be in the middle. This algorithm will try at most 5 times (it gets to resolution differences of ~0.05, which is fair).

To calculate the ARI, we need to load a dataset, select a number of features, and create the dataframe with seeds as rows (to see varation on clustering / triku) and the methods as columns. Because creating each dataframe take time (there are 70 cells to be filled), we will choose two datasets (DE = 0.01 and 0.025) and two number of features (100 and 500), which show good results in the previous sections. 

In [ ]:
save_dir = os.getcwd() + '/exports/comparisons/'

In [ ]:
min_res, max_res, max_depth = 0.3, 2, 6

In [ ]:
for deprobx in tqdm([0.0065, 0.008, 0.01, 0.016, 0.025, 0.05, 0.1, 0.3]):
    adata_all = sc.read(splatter_dir + f'/splatter_deprob_{deprobx}.loom', sparse=False)
    sc.pp.subsample(adata_all, 0.4) # We shorthen this to make the calculations not take 8 ours!
    sc.pp.filter_genes(adata_all, min_cells=1)
    sc.pp.filter_cells(adata_all, min_genes=1)
    sc.pp.log1p(adata_all)
    
    for n_features in tqdm([100, 250, 500]):
        print(deprobx, n_features)
        if not (os.path.exists(os.getcwd() + f'/exports/comparisons/ARI_scatter_{deprobx}_n_features_{n_features}.csv') & 
                os.path.exists(os.getcwd() + f'/exports/comparisons/NMI_scatter_{deprobx}_n_features_{n_features}.csv')):
            df_feature_ranks = pd.read_csv(os.getcwd() + '/exports/comparisons/' + f'scatter_{deprobx}' + '_feature_ranks.csv', index_col=0)

            list_methods = ['triku'] + [i for i in df_feature_ranks.columns if not i.startswith('triku')] + ['all', 'random']

            df_ARI = pd.DataFrame(index=[f'seed_{i}' for i in range(10)], columns=list_methods)
            df_NMI = pd.DataFrame(index=[f'seed_{i}' for i in range(10)], columns=list_methods)

            for seed in tqdm(range(10)):
                for method in tqdm(list_methods):
                    if method.startswith('triku'):
                        feats = df_feature_ranks[f'triku_{seed}'].sort_values().index[:n_features]
                    
                    elif method == "all":
                        feats = df_feature_ranks[f'triku_{seed}'].sort_values().index[:]
                    elif method == "random":
                        array_selection = np.array([False] * len(df_feature_ranks))
                        array_selection[np.random.choice(np.arange(len(df_feature_ranks)), n_features, replace=False)] = True
                        
                        feats = df_feature_ranks[f'triku_{seed}'].sort_values().index[array_selection]
                    
                    else:
                        feats = df_feature_ranks[method].sort_values().index[:n_features]
                    
                    adata_groups = [i.replace('Group', '') for i in adata_all.obs['Group']]
                    c_f, res = clustering_binary_search(adata_all, min_res, max_res, max_depth, seed, len(list(dict.fromkeys(adata_groups))), feats, apply_log=False)
                    ARI = ARS(c_f, adata_groups)
                    NMS = NMI(c_f, adata_groups)

                    df_ARI.loc[f'seed_{seed}', method] = ARI
                    df_NMI.loc[f'seed_{seed}', method] = NMS
            
            print(os.getcwd() + f'/exports/comparisons/NMI_scatter_{deprobx}_n_features_{n_features}.csv')
            df_ARI.to_csv(os.getcwd() + f'/exports/comparisons/ARI_scatter_{deprobx}_n_features_{n_features}.csv')
            df_NMI.to_csv(os.getcwd() + f'/exports/comparisons/NMI_scatter_{deprobx}_n_features_{n_features}.csv')
            

In [ ]:
list_deprobs = [0.0065, 0.008, 0.01, 0.016, 0.025, 0.05, 0.1, 0.3]

In [ ]:
for n_feats in ['100', '250', '500']:
    list_files = [f'ARI_scatter_{deprob}_n_features_{n_feats}.csv' for deprob in list_deprobs[::-1]]
    plot_lab_org_comparison_scores(f'ARI-{n_feats}', '', save_dir, [''], increasing=0, mode='ARI', list_files=list_files, 
                                       title=f'ARI on artificial datasets, {n_feats} features', 
                                       filename=f'ARI_{n_feats}-features')

for n_feats in ['100', '250', '500']:
    list_files = [f'NMI_scatter_{deprob}_n_features_{n_feats}.csv' for deprob in list_deprobs[::-1]]
    plot_lab_org_comparison_scores(f'NMI-{n_feats}', '', save_dir, [''], increasing=0, mode='NMI', list_files=list_files, 
                                       title=f'NMI on artificial datasets, {n_feats} features', 
                                       filename=f'NMI_{n_feats}-features')

For lower number of features (100 - 250) scanpy performs best at lower DE probabilities (up to 0.025) but performs worse at full resolution (0.1 or 0.3), with scry the best method for that, principally because the features that make smaller clusters separate are the ones with most expression, and those are the ones selected by scry. However, at smaller DE probabilities, the features that separate the dataset the most are the ones with mid expression levels, which are best picked by triku. 

In [ ]:
df_values = pd.read_csv(f'{save_dir}/scatter_0.01_feature_values.csv', index_col=0)
df_ranks = pd.read_csv(f'{save_dir}/scatter_0.01_feature_ranks.csv', index_col=0)

In [ ]:
adata = sc.read(splatter_dir + f'/splatter_deprob_0.01.loom', sparse=False)
sc.pp.subsample(adata, 0.4) # We shorthen this to make the calculations not take 8 ours!

adata.raw = adata
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
df_bokeh = pd.DataFrame({'m': np.log10(adata.X.mean(0)), 
                         'z': df['triku_0'].loc[adata.var_names].values, 
                         'n': df.index.values})
                   
p = figure(tools="box_zoom,hover,reset", plot_height=600, plot_width=600, tooltips=[("Gene","@n")])
p.scatter('m', 'z', source=df_bokeh, alpha=0.7, line_color=None)
show(p)

In [ ]:
triku_hvg = df_ranks['triku_0'].values < 250
scry_hvg = df_ranks['scry'].values < 250
std_hvg = df_ranks['std'].values < 250
scanpy_hvg = df_ranks['scanpy'].values < 250

fig, axs = plt.subplots(1, 4, figsize=(12, 3))
names = ['triku', 'scanpy', 'std', 'scry',]
hvgs = [triku_hvg, scanpy_hvg, std_hvg, scry_hvg]
colors = ['#e73f74', '#7f3c8d', '#11a579', '#3969ac']

for i in range(4):
    axs[i].scatter(np.log10(adata.X.mean(0))[~hvgs[i]][::5], 
                   df['triku_0'].loc[adata.var_names].values[~hvgs[i]][::5], c="#dedede", s=2, alpha=0.7)
    axs[i].scatter(np.log10(adata.X.mean(0))[hvgs[i]], 
                   df['triku_0'].loc[adata.var_names].values[hvgs[i]], c=colors[i], s=2, label=names[i])
    axs[i].legend()

fig.text(0.0, 0.5, 'Wasserstein distance', va='center', rotation='vertical')
fig.text(0.5, 0.0, 'log$_{10}$ mean expression', va='center', rotation='horizontal')
plt.tight_layout()
plt.savefig(os.getcwd() + f'/figures/comparison_figs/pdf/barplots_scatter.pdf', fmt='pdf')


list_list_genes = [['Gene1118', 'Gene8599', 'Gene1513', 'Gene1479'],     # triku only
                   ['Gene6723', 'Gene6625', 'Gene9796', 'Gene935'],      # triku + scanpy
                   ['Gene12841', 'Gene10739', 'Gene6729', 'Gene12240'],  # scanpy
                   ['Gene9545', 'Gene4459', 'Gene383', 'Gene12455'],     # all
                   ['Gene1633', 'Gene10792', 'Gene2496', 'Gene12497']]   # std + scry

list_bar_colors = ['#94346E', '#E17C05', '#0F8554', '#1D6996']

for lg_idx, list_genes in enumerate(list_list_genes):
    fig, axs = plt.subplots(1, 5, figsize=(3*5, 3))
    
    hvg = np.isin(adata.var_names, list_genes)
    axs[0].scatter(np.log10(adata.X.mean(0))[~hvg][::5], 
                   df['triku_0'].loc[adata.var_names].values[~hvg][::5], c="#bcbcbc", s=2, alpha=0.7)
    
    for i in range(1, 5):
        for group in range(10):
            axs[0].scatter(np.log10(adata.X.mean(0))[np.isin(adata.var_names, list_genes[i - 1])][::5], 
                           df['triku_0'].loc[adata.var_names].values[np.isin(adata.var_names, list_genes[i - 1])][::5], c=list_bar_colors[i - 1], s=7)

                
            data_values = adata[adata.obs['Group'] == 'Group' + str(group + 1)].X[:, np.argwhere(adata.var_names == list_genes[i-1])[0]].flatten()
            mean, std = np.mean(data_values), np.std(data_values)

            axs[i].bar(group + 1, mean, color=list_bar_colors[i - 1])
            
    axs[0].set_ylabel('Wasserstein distance')
    axs[0].set_xlabel('log$_{10}$ mean expression')
    axs[1].set_ylabel('Mean group expression')
    axs[1].set_xlabel('Group')

    plt.tight_layout()

    plt.savefig(os.getcwd() + f'/figures/comparison_figs/pdf/barplots_{lg_idx}.pdf', fmt='pdf')

# Ding et al. / Mereu et al. datasets
Now that we have seen that triku outperforms other methods in artificial datasets, at least when there is intrinsic noisiness, we are going to apply similar metrics to biological datasets. We are first going to use Mereu's and Ding's human + mouse benchmarking datasets. They will help us see biases on performance of all the methods, and also it will act as a validation of the results from the original papers.

In this part, due to the large amount of datasets, and also due to the heterogeneity of genes, we will not apply use different number of features. Instead, we will run triku with seed 0, and select the default number of features that is automatically generated to select the features on the rest of methods. This will mean that different datasets will have different number of features, although each dataset will have the same number of features across methods. 

The two main methods that we will use to evaluate the feature selection are ARI and Silhouette scores.
* ARI uses the assigned cell types from the paper (Mereu et al. use MatchSCore2 and Ding et al. uses a custom algorithm) and applies the same binary search for resolution.
* Silhouette. It is used in two forms:
    * Apply the same resolution to all datasets and all methods using the binary search, and get the Silhouette from there.
    * Apply Silhouette to the benchmark-assigned cell types.

## Create feature ranking dataframes

In [ ]:
mereu_dir = os.path.dirname(os.getcwd()) + '/data/Mereu_2020/'

for libprep in tqdm(['10X', 'CELseq2', 'ddSEQ', 'Dropseq', 'inDrop', 'QUARTZseq', 'SingleNuclei', 'SMARTseq2']):
    for org in ['human', 'mouse']:
        if os.path.exists(save_dir + f'mereu_{libprep}_{org}_feature_values.csv'):
            print(f'{libprep}, {org} exists!')
        else:
            adata_libprep = sc.read(mereu_dir + f'{libprep}_{org}.h5ad')
            create_df_feature_ranking(adata_libprep, f'mereu_{libprep}_{org}', apply_log=False)

In [ ]:
ding_dir = os.path.dirname(os.getcwd()) + '/data/Ding_2020/'

for libprep in tqdm(['10X', 'CELseq2', 'Dropseq', 'inDrop', 'sci-RNA-seq', 'Seq-Well', 'SingleNuclei', 'SMARTseq2']):
    for org in ['human', 'mouse']:
        if os.path.exists(ding_dir + f'{libprep}_{org}.h5ad'):
            if os.path.exists(save_dir + f'ding_{libprep}_{org}_feature_values.csv'):
                print(f'{libprep}, {org} exists!')
            else:
                adata_libprep = sc.read(ding_dir + f'{libprep}_{org}.h5ad')
                create_df_feature_ranking(adata_libprep, f'ding_{libprep}_{org}', apply_log=False)

## Calculate scores

In [ ]:
@ray.remote
def run_ARI_silhouette_rem(lib_prep, org, seed, lab, adata_dir, save_dir):
    if os.path.exists(adata_dir + f'{lib_prep}_{org}.h5ad'):
        if os.path.exists(save_dir + f'{lab}_{lib_prep}_{org}_comparison-scores_seed-{seed}.csv'):
            print(f'{lib_prep}, {org}, {seed} exists!')
        else:
            adata = sc.read_h5ad(adata_dir + f'{lib_prep}_{org}.h5ad')
            cell_type = 'cell_types' if 'cell_types' in adata.obs else 'CellType' # Somwhere I've fucked up with column name. Don't care where honestly.
            df_rank = pd.read_csv(os.getcwd() + f'/exports/comparisons/{lab}_{lib_prep}_{org}_feature_ranks.csv', index_col=0)

            biological_silhouette_ARI_table(adata, df_rank, outdir=save_dir, file_root=f'{lab}_{lib_prep}_{org}', seed=seed, 
                                                        cell_types_col=cell_type, n_procs=1)         

In [ ]:
# Mereu's datasets
save_dir = os.getcwd() + '/exports/comparisons/'
adata_dir = data_dir + 'Mereu_2020/'


lib_preps = ['SingleNuclei', 'Dropseq', 'inDrop', '10X', 'SMARTseq2', 'CELseq2', 'QUARTZseq']
orgs = ['mouse', 'human'] 
result = list(product(*[lib_preps, orgs, range(5)]))

ray.init(ignore_reinit_error=True)

list_id = [run_ARI_silhouette_rem.remote(lib_prep, org, seed, 'mereu', adata_dir, save_dir) for lib_prep, org, seed in result]
list_results = ray.get(list_id)

ray.shutdown()

In [ ]:
# Ding's datasets
save_dir = os.getcwd() + '/exports/comparisons/'
adata_dir = data_dir + 'Ding_2020/'


lib_preps = ['10X', 'CELseq2', 'Dropseq', 'inDrop', 'sci-RNAseq', 'Seq-Well', 'SingleNuclei', 'SMARTseq2']
orgs = ['mouse', 'human'] 
result = list(product(*[lib_preps, orgs, range(5)]))

ray.init(ignore_reinit_error=True)

list_id = [run_ARI_silhouette_rem.remote(lib_prep, org, seed, 'ding', adata_dir, save_dir) for lib_prep, org, seed in result]
list_results = ray.get(list_id)

ray.shutdown()

In [ ]:
plot_lab_org_comparison_scores('mereu', org='-', read_dir=save_dir, variables=['NMI'], figsize=(16, 4), title=f'NMI on mereu datasets', 
                                  filename=f'mereu-NMI', increasing=False, lognames=1)

In [ ]:
for lab in ['mereu', 'ding']:
    save_dir = os.getcwd() + '/exports/comparisons/'
    log = 1
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['NMI'], figsize=(16, 4), title=f'NMI on {lab} datasets', 
                                  filename=f'{lab}-NMI', increasing=False, lognames=log)
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['ARI'], figsize=(16, 4), title=f'ARI on {lab} datasets', 
                                  filename=f'{lab}-ARI', increasing=False, lognames=log)
    
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_bench_UMAP'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on UMAP projection',
                                   filename=f'{lab}-silhouette_UMAP_celltypes', increasing=False, lognames=log)
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_leiden_UMAP'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on UMAP projection', 
                                  filename=f'{lab}-silhouette_UMAP_leiden', increasing=False, lognames=log)
    
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_bench_PCA'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on PCA projection', 
                                  filename=f'{lab}-silhouette_PCA_celltypes', increasing=False, lognames=log)
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_leiden_PCA'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on PCA projection', 
                                  filename=f'{lab}-silhouette_PCA_leiden', increasing=False, lognames=log)
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['DavBo_bench_PCA'], figsize=(16, 4), 
                                   title=f'Davies-Bouldin on {lab} datasets, cell types on PCA projection [less is better]', 
                                  filename=f'{lab}-Davies-Bouldin_PCA_celltypes', increasing=True, lognames=log)
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['DavBo_leiden_PCA'], figsize=(16, 4), 
                                   title=f'Davies-Bouldin on {lab} datasets, leiden clusters on PCA projection [less is better]', 
                                  filename=f'{lab}-Davies-Bouldin_PCA_leiden', increasing=True, lognames=log)
    

    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_bench_all_hvg'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on selected features',
                                   filename=f'{lab}-silhouette_selected features_celltypes', increasing=False, lognames=log)
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_leiden_all_hvg'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on selected features', 
                                  filename=f'{lab}-silhouette_selected features_leiden', increasing=False, lognames=log)
    
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['DavBo_bench_all_hvg'], figsize=(16, 4), 
                                   title=f'Davies-Bouldin on {lab} datasets, cell types on selected features',
                                   filename=f'{lab}-Davies-Bouldin_selected features_celltypes', increasing=True, lognames=log)
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['DavBo_leiden_all_hvg'], figsize=(16, 4), 
                                   title=f'Davies-Bouldin on {lab} datasets, leiden clusters on selected features', 
                                  filename=f'{lab}-Davies-Bouldin_selected features_leiden', increasing=True, lognames=log)

We see a clear trend in the results. For all metrics (NMI, ARI, Silhouette and Davies-Bouldin) there are two clear groups regarding performance: on the one hand std + scry + brennecke + sscanpy, where the method rely directly or indirectly in the calculation of a variance metric; and triku + nbumi + m3drop, which do not. The methods relying on a metric perform much worse than those which do not. In some cases scanpy's performance is better, comparable to triku + m3drop + nbumi, but oftentimes the metric appears half both types of methods.

Regarding triku's performance, it is at the same level at m3drop and nbumi's. In certain cases, like silhouette scores for 10x human/mouse, Celseq2 mouse and QUARTZseq human, the scores are worse than for nbumi + m3drop, although ARI and NMI have comparabel values. In many other cases, like most of Ding datasets, and Mereu's Dropseq, Smartseq2 and inDrop, Silhouette scores are highest on triku.

# Ding et al. / Mereu et al. datasets (this time with log)
Now that we have seen that triku outperforms other methods in artificial datasets, at least when there is intrinsic noisiness, we are going to apply similar metrics to biological datasets. We are first going to use Mereu's and Ding's human + mouse benchmarking datasets. They will help us see biases on performance of all the methods, and also it will act as a validation of the results from the original papers.

In this part, due to the large amount of datasets, and also due to the heterogeneity of genes, we will not apply use different number of features. Instead, we will run triku with seed 0, and select the default number of features that is automatically generated to select the features on the rest of methods. This will mean that different datasets will have different number of features, although each dataset will have the same number of features across methods. 

The two main methods that we will use to evaluate the feature selection are ARI and Silhouette scores.
* ARI uses the assigned cell types from the paper (Mereu et al. use MatchSCore2 and Ding et al. uses a custom algorithm) and applies the same binary search for resolution.
* Silhouette. It is used in two forms:
    * Apply the same resolution to all datasets and all methods using the binary search, and get the Silhouette from there.
    * Apply Silhouette to the benchmark-assigned cell types.

## Create feature ranking dataframes

In [ ]:
save_dir = os.getcwd() + '/exports/comparisons/'

In [ ]:
mereu_dir = os.path.dirname(os.getcwd()) + '/data/Mereu_2020/'

for libprep in tqdm(['10X', 'CELseq2', 'ddSEQ', 'Dropseq', 'inDrop', 'QUARTZseq', 'SingleNuclei', 'SMARTseq2']):
    for org in ['human', 'mouse']:
        if os.path.exists(save_dir + f'mereu_{libprep}_{org}-log_feature_values.csv'):
            print(f'{libprep}, {org} exists!')
        else:
            adata_libprep = sc.read(mereu_dir + f'{libprep}_{org}.h5ad')
            create_df_feature_ranking(adata_libprep, f'mereu_{libprep}_{org}-log', apply_log=True)

In [ ]:
ding_dir = os.path.dirname(os.getcwd()) + '/data/Ding_2020/'

for libprep in tqdm(['10X', 'CELseq2', 'Dropseq', 'inDrop', 'sci-RNA-seq', 'Seq-Well', 'SingleNuclei', 'SMARTseq2']):
    for org in ['human', 'mouse']:
        if os.path.exists(ding_dir + f'{libprep}_{org}.h5ad'):
            if os.path.exists(save_dir + f'ding_{libprep}_{org}-log_feature_values.csv'):
                print(f'{libprep}, {org} exists!')
            else:
                adata_libprep = sc.read(ding_dir + f'{libprep}_{org}.h5ad')
                create_df_feature_ranking(adata_libprep, f'ding_{libprep}_{org}-log', apply_log=True)

## Calculate scores

In [ ]:
@ray.remote
def run_ARI_silhouette_rem(lib_prep, org, seed, lab, adata_dir, save_dir):
    if os.path.exists(adata_dir + f'{lib_prep}_{org}.h5ad'):
#         if os.path.exists(save_dir + f'{lab}_{lib_prep}-log_{org}_comparison-scores_seed-{seed}.csv'):
#             print(f'{lib_prep}, {org}, {seed} exists!')
#         else:
            adata = sc.read_h5ad(adata_dir + f'{lib_prep}_{org}.h5ad')
            cell_type = 'cell_types' if 'cell_types' in adata.obs else 'CellType' # Somwhere I've fucked up with column name. Don't care where honestly.
            df_rank = pd.read_csv(os.getcwd() + f'/exports/comparisons/{lab}_{lib_prep}_{org}-log_feature_ranks.csv', index_col=0)

            biological_silhouette_ARI_table(adata, df_rank, outdir=save_dir, file_root=f'{lab}_{lib_prep}_{org}-log', seed=seed, 
                                                        cell_types_col=cell_type, n_procs=1)         

In [ ]:
# Mereu's datasets
save_dir = os.getcwd() + '/exports/comparisons/'
adata_dir = data_dir + 'Mereu_2020/'


lib_preps = ['SingleNuclei', 'Dropseq', 'inDrop', '10X', 'SMARTseq2', 'CELseq2', 'QUARTZseq'] 
orgs = ['mouse', 'human'] 
result = list(product(*[lib_preps, orgs, range(5)]))

ray.init(ignore_reinit_error=True, num_cpus=8)

list_id = [run_ARI_silhouette_rem.remote(lib_prep, org, seed, 'mereu', adata_dir, save_dir) for lib_prep, org, seed in result]
list_results = ray.get(list_id)

ray.shutdown()

In [ ]:
# Ding's datasets
save_dir = os.getcwd() + '/exports/comparisons/'
adata_dir = data_dir + 'Ding_2020/'


lib_preps = ['10X', 'CELseq2', 'Dropseq', 'inDrop', 'sci-RNAseq', 'Seq-Well', 'SingleNuclei', 'SMARTseq2']
orgs = ['mouse', 'human'] 
result = list(product(*[lib_preps, orgs, range(5)]))

ray.init(ignore_reinit_error=True, num_cpus=8)

list_id = [run_ARI_silhouette_rem.remote(lib_prep, org, seed, 'ding', adata_dir, save_dir) for lib_prep, org, seed in result]
list_results = ray.get(list_id)

ray.shutdown()

In [ ]:
for lab in ['ding', 'mereu']:
    save_dir = os.getcwd() + '/exports/comparisons/'
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['NMI'], figsize=(16, 4), title=f'NMI on {lab} datasets (log)', 
                                  filename=f'{lab}-NMI-log')
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['ARI'], figsize=(16, 4), title=f'ARI on {lab} datasets (log)', 
                                  filename=f'{lab}-ARI-log')
    
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['Sil_bench_UMAP'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on UMAP projection (log)',
                                   filename=f'{lab}-silhouette_UMAP_celltypes-log')
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['Sil_leiden_UMAP'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on UMAP projection (log)', 
                                  filename=f'{lab}-silhouette_UMAP_leiden-log')
    
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['Sil_bench_PCA'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on PCA projection (log)', 
                                  filename=f'{lab}-silhouette_PCA_celltypes-log')
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['Sil_leiden_PCA'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on PCA projection (log)', 
                                  filename=f'{lab}-silhouette_PCA_leiden-log')
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['DavBo_bench_PCA'], figsize=(16, 4), 
                                   title=f'Davies-Bouldin on {lab} datasets, cell types on PCA projection [less is better] (log)', 
                                  filename=f'{lab}-Davies-Bouldin_PCA_celltypes-log', increasing=True)
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['DavBo_leiden_PCA'], figsize=(16, 4), 
                                   title=f'Davies-Bouldin on {lab} datasets, leiden clusters on PCA projection [less is better] (log)', 
                                  filename=f'{lab}-Davies-Bouldin_PCA_leiden-log', increasing=True)
    
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['Sil_bench_all_hvg'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on selected features (log)',
                                   filename=f'{lab}-silhouette_selected features_celltypes-log')
    plot_lab_org_comparison_scores(lab, org='-log', read_dir=save_dir, variables=['Sil_leiden_all_hvg'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on selected features (log)', 
                                  filename=f'{lab}-silhouette_selected features_leiden-log')

In [ ]:
for lab in ['ding', 'mereu']:
    save_dir = os.getcwd() + '/exports/comparisons/'
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['NMI'], figsize=(16, 4), title=f'NMI on {lab} datasets (log and nonlog)', 
                                  filename=f'{lab}-NMI-both')
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['ARI'], figsize=(16, 4), title=f'ARI on {lab} datasets (log and nonlog)', 
                                  filename=f'{lab}-ARI-both')
    
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_bench_UMAP'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on UMAP projection (log and nonlog)',
                                   filename=f'{lab}-silhouette_UMAP_celltypes-both')
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_leiden_UMAP'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on UMAP projection (log and nonlog)', 
                                  filename=f'{lab}-silhouette_UMAP_leiden-both')
    
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_bench_PCA'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on PCA projection (log and nonlog)', 
                                  filename=f'{lab}-silhouette_PCA_celltypes-both')
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_leiden_PCA'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on PCA projection (log and nonlog)', 
                                  filename=f'{lab}-silhouette_PCA_leiden-both')
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['DavBo_bench_PCA'], figsize=(16, 4), 
                                   title=f'Davies-Bouldin on {lab} datasets, cell types on PCA projection [less is better] (log and nonlog)', 
                                  filename=f'{lab}-Davies-Bouldin_PCA_celltypes-both', increasing=True)
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['DavBo_leiden_PCA'], figsize=(16, 4), 
                                   title=f'Davies-Bouldin on {lab} datasets, leiden clusters on PCA projection [less is better] (log and nonlog)', 
                                  filename=f'{lab}-Davies-Bouldin_PCA_leiden-both', increasing=True)
    
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_bench_all_hvg'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, cell types on selected features (log and nonlog)',
                                   filename=f'{lab}-silhouette_selected features_celltypes-both')
    plot_lab_org_comparison_scores(lab, org='-', read_dir=save_dir, variables=['Sil_leiden_all_hvg'], figsize=(16, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on selected features (log and nonlog)', 
                                  filename=f'{lab}-silhouette_selected features_leiden-both')

We see that, generally, all types of scores are unaltered by log transformation. Interestingly, in some datasets (ding SMARTseq2 human, Seq-Well human, 10X mouse; mereu inDrop mouse, CELseq2 human, SingleNuclei human) either NMI or Davies-Bouldin / Silhouette scores worsen in nbumi and/or brennecke methods. Regarding triku, log-transformation keeps the scores unaltered. Therefore, we can safely apply triku in either log-transformed or raw data, and cluster metric will remain the same.

In [ ]:
for lab in ['mereu', 'ding']:
    list_files = [i for i in os.listdir(save_dir) if lab in i and "comparison-scores" in i and 'log' not in i]
    list_files_log = [i for i in os.listdir(save_dir) if lab in i and "comparison-scores" in i and 'log' in i]

    for var, title in zip(
        ['ARI', 'NMI', 'Sil_bench_UMAP', 'Sil_leiden_UMAP', 'Sil_bench_PCA', 'Sil_leiden_PCA', 'Sil_bench_all_hvg', 'Sil_leiden_all_hvg'], 
        ['Rank comparison of ' + i for i in 
         ['ARI', 'NMI', 'Silhouette of cell types in UMAP', 'Silhouette of leiden clusters in UMAP', 
          'Silhouette of cell types in PCA', 'Silhouette of leiden clusters in PCA', 
          'Silhouette of cell types in selected features', 'Silhouette of leiden clusters in selected features']]):
        
        
        compare_rankings(
            list_files,
            list_files_log,
            read_dir=save_dir,
            title=f'{title} - {lab}',
            title1="Non log",
            title2="Log",
            variables=[var],
            increasing=False,
            alpha=0.05,
            figsize=(4, 5),
            mode="normal",
            filename=f"rank_comparison-{lab}-{title.replace(' ', '_')}"
            )

In [ ]:
for lab in ['mereu', 'ding']:
    list_files = sorted([i for i in os.listdir(save_dir) if lab in i and "comparison-scores" in i and 'log' not in i])
    list_files_log = sorted([i for i in os.listdir(save_dir) if lab in i and "comparison-scores" in i and 'log' in i])

    for var, title in zip(
        ['ARI', 'NMI', 'Sil_bench_UMAP', 'Sil_leiden_UMAP', 'Sil_bench_PCA', 'Sil_leiden_PCA', 'Sil_bench_all_hvg', 'Sil_leiden_all_hvg'], 
        ['log - nonlog differences on ' + i for i in 
         ['ARI', 'NMI', 'Silhouette of cell types in UMAP', 'Silhouette of leiden clusters in UMAP', 
          'Silhouette of cell types in PCA', 'Silhouette of leiden clusters in PCA', 
          'Silhouette of cell types in selected features', 'Silhouette of leiden clusters in selected features']]):
                
        compare_values(
            list_files_log,
            list_files,
            read_dir=save_dir,
            title=f"{title} - {lab}",
            variables=[var],
            figsize=(10, 4),
            filename=f"difference-{lab}-{title.replace(' ', '_')}"
            )

## Explainability of results
Although we see that triku has promising results, we were striked at how std, scry and brennecke have such a big gap of scores with respect to triku, nbumi and m3drop. In this section we are going to apply some comprobation measures to see if we can know why the difference is so big. 

Some comprobations we are going to do are:
* Plot UMAPs of datasets with observed cell types and leiden solutions.
* Plot CV^2 / std / percentage of zeros VS mean plots to see biases on each method.
* Plot heatmaps with percentage of overlap between pairs of techniques to see similarities. We expect triku + nbumi + m3drop to have higher similarity and brennecke + std + scry also to be similar together.

### UMAPs

In [ ]:
exports_dir = os.getcwd() + '/exports/comparisons/'

In [ ]:
# WARNING!!! create_UMAP_adataset_libprep_org has harcoded feature selection method names!!
dict_UMAP_info_mereu = create_dict_UMAPs_datasets(adata_dir=data_dir + 'Mereu_2020/', df_rank_dir=exports_dir, lab='mereu', 
                                                  lib_preps=['SingleNuclei', 'Dropseq', 'inDrop', '10X', 'SMARTseq2', 'CELseq2', 'QUARTZseq'], 
                                                  list_orgs = ['human', 'mouse'], log=True)

f = open(f"{exports_dir}/dict_info_mereu.pkl","wb")
pickle.dump(dict_UMAP_info_mereu,f)
f.close()

In [ ]:
dict_UMAP_info_ding = create_dict_UMAPs_datasets(adata_dir=data_dir + 'Ding_2020/', df_rank_dir=exports_dir, lab='ding', 
                                                  lib_preps=['10X', 'CELseq2', 'Dropseq', 'inDrop', 'sci-RNAseq', 'Seq-Well', 'SingleNuclei', 'SMARTseq2'], 
                                                  list_orgs = ['human', 'mouse'], log=True)

f = open(f"{exports_dir}/dict_info_ding.pkl","wb")
pickle.dump(dict_UMAP_info_ding,f)
f.close()

In [ ]:
with open(f"{exports_dir}/dict_info_mereu.pkl", 'rb') as f:
    dict_UMAP_info_mereu = pickle.load(f)

with open(f"{exports_dir}/dict_info_ding.pkl", 'rb') as f:
    dict_UMAP_info_ding = pickle.load(f)

In [ ]:
plot_UMAPs_datasets(dict_UMAP_info_mereu, os.getcwd() + '/figures/comparison_figs', lab='mereu', figsize=(20, 35))

In [ ]:
plot_UMAPs_datasets(dict_UMAP_info_ding, os.getcwd() + '/figures/comparison_figs', lab='ding', figsize=(20, 25))

In [ ]:
def plot_XY(dict_returns, x_var, y_var, fig_save_dir, lab, figsize=(20, 35), logx=True, logy=True, title=''):
    list_rows = list(dict_returns.keys())
    list_methods = list(dict_returns[list_rows[0]].keys())[:-1]
        
    fig, axs = plt.subplots(len(list_rows), len(list_methods), figsize=figsize)
    
    for row_idx, row_name in enumerate(list_rows):
        for col_idx, col_name in enumerate(list_methods):
            x_coords = dict_returns[row_name]['other_stuff'][x_var]
            y_coords = dict_returns[row_name]['other_stuff'][y_var]
            highly_variable = dict_returns[row_name][col_name]['highly_variable']
            
            if logx: 
                x_coords = np.log10(x_coords)
            if logy: 
                y_coords = np.log10(y_coords)
            
            axs[row_idx][col_idx].scatter(x_coords[highly_variable == False], y_coords[highly_variable == False], c = '#cbcbcb', alpha=0.05, s=2)
            axs[row_idx][col_idx].scatter(x_coords[highly_variable == True], y_coords[highly_variable == True], c = '#007ab7', alpha=0.2, s=2)
                       
            if row_idx == 0:
                axs[row_idx][col_idx].set_title(f"{col_name}")
                
            if col_idx == 0:
                axs[row_idx][col_idx].set_ylabel(f"{row_name}".replace(' ', '\n'))
                axs[row_idx][col_idx].yaxis.set_label_position('left') 
    
    fig.suptitle(title)
    plt.tight_layout()
    
    for fmt in ['png', 'svg']:
        os.makedirs(f'{fig_save_dir}/{fmt}', exist_ok=True)
        fig.savefig(f'{fig_save_dir}/{fmt}/{lab}_{x_var}-VS-{y_var}.{fmt}', bbox_inches='tight')
        
    plt.show()

In [ ]:
plot_XY(dict_UMAP_info_mereu, 'mean', 'per_0', lab='mereu', fig_save_dir=os.getcwd() + '/figures/comparison_figs', logx=True, logy=False, 
       title='Mereu, percentage of zeros VS log mean')

In [ ]:
plot_XY(dict_UMAP_info_mereu, 'mean', 'std', lab='mereu', fig_save_dir=os.getcwd() + '/figures/comparison_figs', logx=True, logy=True, 
       title='Mereu, log std VS log mean')

In [ ]:
plot_XY(dict_UMAP_info_mereu, 'mean', 'CV2', lab='mereu', fig_save_dir=os.getcwd() + '/figures/comparison_figs', logx=True, logy=True, 
       title='Mereu, log CV2 VS log mean')

In [ ]:
plot_XY(dict_UMAP_info_mereu, 'mean', 'disp', lab='mereu', fig_save_dir=os.getcwd() + '/figures/comparison_figs', logx=True, logy=False, 
       title='Mereu, dispersion (scanpy) VS log mean')

In [ ]:
plot_XY(dict_UMAP_info_ding, 'mean', 'per_0', lab='ding', fig_save_dir=os.getcwd() + '/figures/comparison_figs', logx=True, logy=False, 
       title='ding, percentage of zeros VS log mean')

In [ ]:
plot_XY(dict_UMAP_info_ding, 'mean', 'std', lab='ding', fig_save_dir=os.getcwd() + '/figures/comparison_figs', logx=True, logy=True, 
       title='ding, log std VS log mean')

In [ ]:
plot_XY(dict_UMAP_info_ding, 'mean', 'CV2', lab='ding', fig_save_dir=os.getcwd() + '/figures/comparison_figs', logx=True, logy=True, 
       title='ding, log CV2 VS log mean')

In [ ]:
plot_XY(dict_UMAP_info_ding, 'mean', 'disp', lab='ding', fig_save_dir=os.getcwd() + '/figures/comparison_figs', logx=True, logy=False, 
       title='ding, dispersion (scanpy) VS log mean')

In [ ]:
def plot_heatmaps_jaccard(dict_info, size_factors=(5, 5), fig_save_dir='', lab=''):
    import math 
    import seaborn as sns
    
    n_datasets = len(dict_info)
    n_cols = min(4, int(n_datasets ** 0.5))
    n_rows = math.ceil(n_datasets / n_cols)
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_rows * size_factors[0], n_cols * size_factors[1]))
    
    for row in range(n_rows):
        for col in range(n_cols):
            sq_number = col + n_cols * row
            
            if sq_number < n_datasets:
            
                dataset_dict = dict_info[list(dict_info.keys())[sq_number]]

                methods = list(dataset_dict.keys())[:-1]

                array_heatmap = np.empty((len(methods), len(methods)))
                array_heatmap[:] = np.NaN

                for i, method_i in enumerate(methods):
                    for j, method_j in enumerate(methods):
                        if i >= j:
                            hvg_i, hvg_j = dataset_dict[method_i]['highly_variable'], dataset_dict[method_j]['highly_variable']
                            jaccard = np.sum(hvg_i & hvg_j)/np.sum(hvg_i | hvg_j)
                            array_heatmap[i, j] = jaccard

                h = sns.heatmap(array_heatmap, cbar=False, ax=axs[row][col], annot=True, xticklabels=methods, yticklabels=methods)
                h.set_title(list(dict_info.keys())[sq_number])
            
            else:
                axs[row][col].set_frame_on(False)
                axs[row][col].get_xaxis().set_visible(False)
                axs[row][col].get_yaxis().set_visible(False)
    
    plt.tight_layout()
    for fmt in ['png', 'pdf']:
        os.makedirs(f'{fig_save_dir}/{fmt}', exist_ok=True)
        fig.savefig(f'{fig_save_dir}/{fmt}/{lab}_heatmap_overlap_features.{fmt}', bbox_inches='tight')
        
    plt.show()

            

In [ ]:
plot_heatmaps_jaccard(dict_UMAP_info_mereu, size_factors=(2.7, 2.7*3), fig_save_dir=os.getcwd() + '/figures/comparison_figs', lab='mereu')

In [ ]:
plot_heatmaps_jaccard(dict_UMAP_info_ding, size_factors=(4.3, 4.3), fig_save_dir=os.getcwd() + '/figures/comparison_figs', lab='ding')

# Enrichment of ribosomal and mitochondrial genes

In [ ]:
def barplot_mt_rbp(lab, org, method, n_features=[100, 250, 500, 1000], mode=0):
    list_FS = ['triku_0', 'scanpy', 'std', 'scry', 'brennecke', 'm3drop', 'nbumi'] # std is missing
    palette = ["#E73F74","#7F3C8D","#11A579","#3969AC","#F2B701","#80BA5A","#E68310","#a0a0a0","#505050"]
    
    fig, axs = plt.subplots(2, 1, figsize=(10, 6))
    
    for n_feature_idx, n_feature in enumerate(n_features):
        for FS_idx, FS in enumerate(list_FS):
            df = pd.read_csv(os.getcwd() + f'/exports/comparisons/{lab}_{method}_{org}-log_feature_ranks.csv', index_col=0)

            set_rbp = set([i for i in df.index if (i.upper().startswith('RPS')) | (i.upper().startswith('RPL'))])
            set_mt = set([i for i in df.index if (i.upper().startswith('MT-'))])
            
            set_FS = set(df.sort_values(by=FS).index.tolist()[:n_feature])
            
            if mode == 0:
                axs[0].bar(n_feature_idx + (FS_idx - len(list_FS) // 2) / (len(list_FS) + 3) , 100 * len(set_rbp & set_FS)/len(set_FS), 
                        width = 0.1, color=palette[FS_idx])
                axs[1].bar(n_feature_idx + (FS_idx - len(list_FS) // 2) / (len(list_FS) + 3) , 100 * len(set_mt & set_FS)/len(set_FS), 
                        width = 0.1, color=palette[FS_idx])
            else:
                axs[0].bar(n_feature_idx + (FS_idx - len(list_FS) // 2) / (len(list_FS) + 3) , 100 * len(set_rbp & set_FS)/len(set_rbp), 
                        width = 0.1, color=palette[FS_idx])
                axs[1].bar(n_feature_idx + (FS_idx - len(list_FS) // 2) / (len(list_FS) + 3) , 100 * len(set_mt & set_FS)/len(set_rbp), 
                        width = 0.1, color=palette[FS_idx])
                
    for ax in axs:
        ax.set_xticks(range(len(n_features)))
        ax.set_xticklabels(n_features)
    
    if mode == 0:
        axs[0].set_ylabel('% ribosomal genes\n(from selected features)')
        axs[1].set_ylabel('% mitochondrial genes\n(from selected features)')
    else:
        axs[0].set_ylabel('% ribosomal genes\n(from all ribosomal genes)')
        axs[1].set_ylabel('% mitochondrial genes\n(from all mitochondrial genes)')
        
    legend_elements = [mpl.lines.Line2D([0], [0], marker="o", color=palette[0], label='triku')] + [
        mpl.lines.Line2D(
            [0], [0], marker="o", color=palette[j], label=list_FS[j]
        )
        for j in range(1, len(list_FS))
    ]
    axs[0].legend(handles=legend_elements, bbox_to_anchor=(1.2, 0.9))
    
    
def heatmap_mt_rbp(labs, orgs, methods, n_features=500):
    list_FS = ['triku_0', 'm3drop', 'nbumi', 'scanpy', 'std', 'scry', 'brennecke', ] # std is missing
    palette = ["#E73F74","#80BA5A","#E68310","#7F3C8D","#11A579","#3969AC","#F2B701","#a0a0a0","#505050"]
        
    dict_info = {}
    
    for lab in labs:
        for org in orgs:
            for method in methods:
                for FS_idx, FS in enumerate(list_FS):
                    if not os.path.exists(os.getcwd() + f'/exports/comparisons/{lab}_{method}_{org}-log_feature_ranks.csv'):
                        continue
                        
                    df = pd.read_csv(os.getcwd() + f'/exports/comparisons/{lab}_{method}_{org}-log_feature_ranks.csv', index_col=0)

                    set_rbp = set([i for i in df.index if (i.upper().startswith('RPS')) | (i.upper().startswith('RPL'))])
                    set_mt = set([i for i in df.index if (i.upper().startswith('MT-'))])

                    set_FS = set(df.sort_values(by=FS).index.tolist()[:n_features])
                    
                    for opt in [f'{FS}_per_rbp_all_features', f'{FS}_per_rbp_rbp_features', 
                                f'{FS}_per_mt_all_features', f'{FS}_per_mt_mt_features']:
                        if opt not in dict_info:
                            dict_info[opt] = []
                    
                    dict_info[f'{FS}_per_rbp_all_features'].append(100 * len(set_rbp & set_FS) / len(set_FS))
                    dict_info[f'{FS}_per_rbp_rbp_features'].append(100 * len(set_rbp & set_FS) / len(set_rbp))
                    dict_info[f'{FS}_per_mt_all_features'].append(100 * len(set_mt & set_FS) / len(set_FS))
                    dict_info[f'{FS}_per_mt_mt_features'].append(100 * len(set_mt & set_FS) / len(set_mt))
    
    df = pd.DataFrame(index=['triku'] + list_FS[1:], columns=[
        'Percentage RBPs in selected features', f'Percentage RBPs from RBP gene list ({len(set_rbp)} elements)', 
        'Percentage MTs in selected features', f'Percentage MTs from MT gene list ({len(set_mt)} elements)'])
    
    for FS_idx, FS in enumerate(list_FS):
        df.iloc[FS_idx, 0] = '%.3f' % np.nanmean(dict_info[f'{FS}_per_rbp_all_features'])
        df.iloc[FS_idx, 1] = '%.3f' % np.nanmean(dict_info[f'{FS}_per_rbp_rbp_features']) 
        df.iloc[FS_idx, 2] = '%.3f' % np.nanmean(dict_info[f'{FS}_per_mt_all_features'])
        df.iloc[FS_idx, 3] = '%.3f' % np.nanmean(dict_info[f'{FS}_per_mt_mt_features'])
                                
                                     
    return df.astype(float)

In [ ]:
df = heatmap_mt_rbp(['ding'], ['human', 'mouse'], ['SingleNuclei', 'Dropseq', 'inDrop', '10X', 'SMARTseq2', 
                                              'CELseq2', 'QUARTZseq', 'sci-RNAseq', 'Seq-Well'], n_features=1000)

display(df)
sns.heatmap(df)

In [ ]:
df = heatmap_mt_rbp(['mereu'], ['human', 'mouse'], ['SingleNuclei', 'Dropseq', 'inDrop', '10X', 'SMARTseq2', 
                                              'CELseq2', 'QUARTZseq', 'sci-RNAseq', 'Seq-Well'], n_features=1000)

display(df)
sns.heatmap(df)

In [ ]:
lab, org, method, n_features = 'ding', 'human', 'Dropseq', [100, 250, 500, 1000]

barplot_mt_rbp(lab, org, method, n_features=[100, 250, 500, 1000], mode=0)
barplot_mt_rbp(lab, org, method, n_features=[100, 250, 500, 1000], mode=1)

# Gene ontology analysis

To see which method is better, a possible idea is to run Enrichr with the selected features, and use it to compare the FS methods. If the ontologies from one method have better p-values/scores, it is likely that they are more representative of the dataset.


In [ ]:
import gseapy

In [ ]:
os.makedirs(os.getcwd() + f'/exports/enrichr/', exist_ok=True)

In [ ]:
list_onto_mouse = ['KEGG_2019_Mouse', 'WikiPathways_2019_Mouse', 'GO_Biological_Process_2018', 'GO_Cellular_Component_2018', 
                   'GO_Molecular_Function_2018',]

list_onto_human = ['KEGG_2019_Human', 'WikiPathways_2019_Human', 'GO_Biological_Process_2018', 'GO_Cellular_Component_2018', 
                   'GO_Molecular_Function_2018', ]

In [ ]:
@ray.remote
def call_enrichr(lab, org, method, n_features, FS):
    if os.path.exists(os.getcwd() + f'/exports/enrichr/{lab}_{method}_{org}_{n_features}_{FS}.csv'):
        print(os.getcwd() + f'/exports/enrichr/{lab}_{method}_{org}_{n_features}_{FS}.csv EXISTS!')
        return None
    
    if not os.path.exists(os.getcwd() + f'/exports/comparisons/{lab}_{method}_{org}-log_feature_ranks.csv'):
        return None
    
    df_file = pd.read_csv(os.getcwd() + f'/exports/comparisons/{lab}_{method}_{org}-log_feature_ranks.csv', index_col=0)
    
    list_genes = df_file.sort_values(by=FS).index.tolist()[:n_features]
    list_onto = list_onto_mouse if org == 'mouse' else list_onto_human
    
    n_trials = 0
    
    while n_trials < 5:
        try:
            result_df = gseapy.enrichr(list_genes, list_onto, cutoff=1, organism=org).results
            if FS == 'triku_0':
                FS = 'triku'
            result_df.to_csv(os.getcwd() + f'/exports/enrichr/{lab}_{method}_{org}_{n_features}_{FS}.csv', index=None)
            n_trials += 10
        except:
            n_trials += 1
            print(f'TRIAL {n_trials}')
            
        

    

In [ ]:
list_comb = list(product(*[['ding', 'mereu'], 
                           ['human', 'mouse'], 
                           ['SingleNuclei', 'Dropseq', 'inDrop', '10X', 'SMARTseq2', 'CELseq2', 'QUARTZseq', 'sci-RNAseq', 'Seq-Well'], 
                           [100, 250, 500, 1000, 1250, 1500], 
                           ['triku_0', 'scanpy', 'std', 'scry', 'brennecke', 'm3drop', 'nbumi']]))


ray.init(ignore_reinit_error=True)

list_id = [call_enrichr.remote(lab, org, method, n_features, FS) for lab, org, method, n_features, FS in list_comb]
list_results = ray.get(list_id)

ray.shutdown()

In [ ]:
def scatter_enrichr(lab, org, method, n_features, n_ontologies=30, column_sort='Adjusted P-value', plot_type='bar', 
                    list_onto=['KEGG_2019_Mouse', 'WikiPathways_2019_Mouse', 'KEGG_2019_Human', 'WikiPathways_2019_Human',
                               'GO_Biological_Process_2018', 'GO_Cellular_Component_2018', 'GO_Molecular_Function_2018',], save=True):
    list_FS = ['triku', 'scanpy', 'std', 'scry', 'brennecke', 'm3drop', 'nbumi']
    palette = ["#E73F74","#7F3C8D","#11A579","#3969AC","#F2B701","#80BA5A","#E68310","#a0a0a0","#505050"]
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 3))
    
    dict_dfs = {}
    
    for n_feature_idx, n_feature in enumerate(n_features):
        for FS_idx, FS in enumerate(list_FS):
            df = pd.read_csv(os.getcwd() + f'/exports/enrichr/{lab}_{method}_{org}_{n_feature}_{FS}.csv')
            df = df[df['Gene_set'].isin(list_onto)]
            
            if column_sort == 'Adjusted P-value':
                df = df.sort_values(by=column_sort).iloc[:n_ontologies]
                y_vals = df[column_sort].values
                y_vals = - np.log10(y_vals)

            elif column_sort == 'Combined Score':
                df = df.sort_values(by=column_sort, ascending=False).iloc[:n_ontologies]
                y_vals = df[column_sort].values
            
            elif column_sort == 'division':
                table_vals = df['Overlap'].values
                df['divided'] = [int(i.split('/')[0]) / int(i.split('/')[1]) for i in table_vals]
                df = df.sort_values(by='divided', ascending=False).iloc[:n_ontologies]
                y_vals = df['divided'].values
                
            
            x_pos = n_feature_idx + (FS_idx - len(list_FS) // 2) / (len(list_FS) + 3)
            
            if plot_type == 'bar':
                plt.bar(x_pos , np.mean(y_vals), 
                        width = 0.1, yerr=np.std(y_vals), color=palette[FS_idx])
            elif plot_type == 'scatter':
                plt.scatter([x_pos] * len(y_vals), y_vals, c=palette[FS_idx], alpha=0.8)
            
            dict_dfs[f'{n_feature}_{FS}'] = df
    
    legend_elements = [
        mpl.lines.Line2D(
            [0], [0], marker="o", color=palette[j], label=list_FS[j]
        )
        for j in range(len(list_FS))
    ]
    ax.legend(handles=legend_elements, bbox_to_anchor=(1.2, 0.9))
    ax.set_xticks(range(len(n_features)))
    ax.set_xticklabels(n_features)
    ax.set_ylabel(column_sort)
    ax.set_xlabel('Number of features')
    
    plt.tight_layout()
    
    if save:
        plt.savefig(save)
    
    return dict_dfs
    

In [ ]:
def barplot_ontologies_individual(df, axis=None, color="#ababab", column='Adjusted P-value', ascending=False, log=True, y_text=''):
    if axis is None:
        fig, axis = plt.subplots(1, 1, figsize=(10, 7))
    
    vals = df.sort_values(by=column, ascending=ascending)[column].values
    names = [i.split(' (')[0] for i in df.sort_values(by=column, ascending=ascending)['Term'].values]
    names = [i[: 42] + '...' if len(i) > 42 else i for i in names]

    if log:
        vals = - np.log10(df.sort_values(by=column, ascending=ascending)[column].values)
    
    if column == 'Adjusted P-value':
        if log:
            axis.plot(-np.log10([0.05, 0.05]), [-1.5, len(names) + 0.5], c="#ababab", alpha=0.8, linewidth=3, zorder=0)
        else:
            axis.plot([0.05, 0.05], [-1.5, len(names) + 0.5], c="#ababab", alpha=0.8, linewidth=3, zorder=0)
        
    axis.barh(range(len(df)), vals, color=color, zorder=5)
    
    for y in range(len(df)):
        axis.text(0.05 * np.max(axis.get_xlim()), y - 0.2, names[y], zorder=10, fontsize=12)
        
    axis.set_yticks([])
    axis.spines['right'].set_visible(False)
    axis.spines['top'].set_visible(False)

    x_text = column if not log else column + ' (log)'
    axis.set_xlabel(x_text)
    axis.set_ylabel(y_text)
    
    return axis


def barplot_ontologies_all(dict_dfs, n_features=1000, list_FSs=['triku', 'std', 'scry', 'scanpy', 'm3drop', 'nbumi'], 
                           list_colors=["#E73F74", "#11A579","#3969AC", "#7F3C8D", "#80BA5A","#E68310"], figsize=(17, 14), save=''):
    
    mpl.rcParams.update({'font.size':17})
    fig, axis = plt.subplots(2, 3, figsize=figsize)
    
    for i in range(len(list_FSs)):
        barplot_ontologies_individual(dict_dfs[f'{n_features}_{list_FSs[i]}'], axis=axis.ravel()[i], 
                                      color=list_colors[i], column='Adjusted P-value', ascending=False, log=True, y_text=list_FSs[i])
    
    plt.tight_layout()
    
    if save:
        plt.savefig(save)
        
    mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
enrichr_figs_dir = os.getcwd() + '/figures/enrichr_figs/'
os.makedirs(enrichr_figs_dir, exist_ok=True)

In [ ]:
# Good in Ding: Dropseq Human (Immune cells)

lab, org, method, n_features = 'ding', 'human', 'Dropseq', [100, 250, 500, 1000, 1250, 1500]
list_dfs_ding_human_dropseq = []
for x in ['Adjusted P-value']:  # ['Combined Score', 'Adjusted P-value', 'division']:
    dict_df = scatter_enrichr(lab, org, method, n_features, n_ontologies=25, column_sort=x, plot_type='scatter', 
                    list_onto=[ 'GO_Biological_Process_2018',], save=enrichr_figs_dir + f'scatter_{lab}_{org}_{method}_{x}.pdf')
    list_dfs_ding_human_dropseq.append(dict_df)
    
    

In [ ]:
barplot_ontologies_all(list_dfs_ding_human_dropseq[0], save=enrichr_figs_dir + f'barplots_{lab}_{org}_{method}_{x}.pdf')

In [ ]:
# Good in Mereu: Dropseq Mouse (Colon cells)

lab, org, method, n_features = 'mereu', 'mouse', 'Dropseq', [100, 250, 500, 1000, 1250, 1500]
list_dfs_mereu_mouse_dropseq = []
for x in ['Adjusted P-value']:  # ['Combined Score', 'Adjusted P-value', 'division']:
    dict_df = scatter_enrichr(lab, org, method, n_features, n_ontologies=25, column_sort=x, plot_type='scatter', 
                    list_onto=['GO_Biological_Process_2018',], save=enrichr_figs_dir + f'scatter_{lab}_{org}_{method}_{x}.pdf')
    list_dfs_mereu_mouse_dropseq.append(dict_df)

In [ ]:
barplot_ontologies_all(list_dfs_mereu_mouse_dropseq[0], save=enrichr_figs_dir + f'barplots_{lab}_{org}_{method}_{x}.pdf')

In [ ]:
# Bad in Ding: 10X Human (Immune cells)

lab, org, method, n_features = 'ding', 'human', '10X', [100, 250, 500, 1000, 1250, 1500]
list_dfs_ding_human_10x = []
for x in ['Adjusted P-value']:  # ['Combined Score', 'Adjusted P-value', 'division']:
    dict_df = scatter_enrichr(lab, org, method, n_features, n_ontologies=25, column_sort=x, plot_type='scatter', 
                    list_onto=['GO_Biological_Process_2018',], save=enrichr_figs_dir + f'scatter_{lab}_{org}_{method}_{x}.pdf')
    list_dfs_ding_human_10x.append(dict_df)

In [ ]:
barplot_ontologies_all(list_dfs_ding_human_10x[0], save=enrichr_figs_dir + f'barplots_{lab}_{org}_{method}_{x}.pdf')

In [ ]:
# Bad in Mereu: 10X Mouse (Colon cells)

lab, org, method, n_features = 'mereu', 'mouse', '10X', [100, 250, 500, 1000, 1250, 1500]
list_dfs_mereu_mouse_10x = []
for x in ['Adjusted P-value']:  # ['Combined Score', 'Adjusted P-value', 'division']:
    dict_df = scatter_enrichr(lab, org, method, n_features, n_ontologies=25, column_sort=x, plot_type='scatter', 
                    list_onto=['GO_Biological_Process_2018',], save=enrichr_figs_dir + f'scatter_{lab}_{org}_{method}_{x}.pdf')
    list_dfs_mereu_mouse_10x.append(dict_df)

In [ ]:
barplot_ontologies_all(list_dfs_mereu_mouse_10x[0], save=enrichr_figs_dir + f'barplots_{lab}_{org}_{method}_{x}.pdf')

# 10X datasets
In this section we are going to analyze the 10x neuron, heart and pbmc datasets.
We are going to show the Silhouette scores for leiden using the methods.

In [ ]:
_10x_dir = os.path.dirname(os.getcwd()) + '/data/10x/'
save_dir = os.getcwd() + '/exports/comparisons/'

In [ ]:
for libprep in tqdm(['neuron', 'pbmc', 'heart', ]):
    if os.path.exists(save_dir + f'10X-datasets_{libprep}_feature_values.csv'):
        print(f'{libprep}, exists!')
    else:
        adata_libprep = sc.read_10x_h5(_10x_dir + f'{libprep}_10k_v3_raw_feature_bc_matrix.h5')
        adata_libprep.var_names_make_unique()
        sc.pp.filter_cells(adata_libprep, min_counts=400)
        sc.pp.filter_genes(adata_libprep, min_counts=100)
        adata_libprep.var_names_make_unique()
        adata_libprep.X = np.asarray(adata_libprep.X.todense())
        del [adata_libprep.var, adata_libprep.obs]

        print(adata_libprep.X)
        create_df_feature_ranking(adata_libprep, f'10X-datasets_nonlog_{libprep}')

In [ ]:
save_dir = os.getcwd() + '/exports/comparisons/'

tissues = ['heart', 'pbmc', 'neuron']

for tissue in tissues:
    print(tissue)
    adata_libprep = sc.read_10x_h5(_10x_dir + f'{tissue}_10k_v3_raw_feature_bc_matrix.h5')
    adata_libprep.var_names_make_unique()
    sc.pp.filter_cells(adata_libprep, min_counts=400)
    sc.pp.filter_genes(adata_libprep, min_counts=100)
    adata_libprep.var_names_make_unique()
    adata_libprep.X = np.asarray(adata_libprep.X.todense())
    df_rank = pd.read_csv(os.getcwd() + f'/exports/comparisons/10X-datasets_nonlog_{tissue}_feature_ranks.csv', index_col=0)
    
    for seed in range(5):
        biological_silhouette_ARI_table(adata_libprep, df_rank, outdir=save_dir, file_root=f'10X-datasets_nonlog_{tissue}', seed=seed, 
                                                            cell_types_col=None, n_procs=1)



In [ ]:
for libprep in tqdm(tissues):
    if os.path.exists(save_dir + f'10X-datasets_log_{libprep}_feature_values.csv'):
        print(f'{libprep}, exists!')
    else:
        adata_libprep = sc.read_10x_h5(_10x_dir + f'{libprep}_10k_v3_raw_feature_bc_matrix.h5')
        adata_libprep.var_names_make_unique()
        sc.pp.filter_cells(adata_libprep, min_counts=400)
        sc.pp.filter_genes(adata_libprep, min_counts=100)
        adata_libprep.var_names_make_unique()
        adata_libprep.X = np.asarray(adata_libprep.X.todense())
        del [adata_libprep.var, adata_libprep.obs]
        
        sc.pp.log1p(adata_libprep)
        
        print(adata_libprep.X)
        create_df_feature_ranking(adata_libprep, f'10X-datasets_log_{libprep}')

In [ ]:
save_dir = os.getcwd() + '/exports/comparisons/'

tissues = ['heart', 'pbmc', 'neuron']

for tissue in tissues:
    print(tissue)
    adata_libprep = sc.read_10x_h5(_10x_dir + f'{tissue}_10k_v3_raw_feature_bc_matrix.h5')
    adata_libprep.var_names_make_unique()
    sc.pp.filter_cells(adata_libprep, min_counts=400)
    sc.pp.filter_genes(adata_libprep, min_counts=100)
    adata_libprep.var_names_make_unique()
    adata_libprep.X = np.asarray(adata_libprep.X.todense())
        
    sc.pp.log1p(adata_libprep)

    df_rank = pd.read_csv(os.getcwd() + f'/exports/comparisons/10X-datasets_log_{tissue}_feature_ranks.csv', index_col=0)
    
    for seed in range(5):
        biological_silhouette_ARI_table(adata_libprep, df_rank, outdir=save_dir, file_root=f'10X-datasets_log_{tissue}', seed=seed, 
                                                            cell_types_col=None, n_procs=1)



In [ ]:
lab = '10X-datasets'

plot_lab_org_comparison_scores(lab=lab, org='-', read_dir=save_dir, variables=['Sil_leiden_PCA'], figsize=(14, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on PCA projection', 
                                  filename=f'{lab}-Silhouette_PCA_leiden')

plot_lab_org_comparison_scores(lab=lab, org='-', read_dir=save_dir, variables=['Sil_leiden_all_hvg'], figsize=(14, 4), 
                                   title=f'Silhouette on {lab} datasets, leiden clusters on selected features', 
                                  filename=f'{lab}-Silhouette_selected features_celltypes')